# Machine Learning 2 Assignment 3

No chatGPT or Copilot was involved in the making at whatsoever.

In [35]:
import pandas
import torch
from torchvision import transforms
from torch.utils.data import Dataset, random_split, DataLoader
import pandas as pd
from PIL import Image
import numpy as np
import glob
import time
import wandb
from torchvision.transforms.functional import to_pil_image
from torch.utils.data.dataset import random_split

In [2]:
class CustomImageDataset(Dataset):

    def read_data(self):
        # Need to insert train csv with file name and correct label 
        train_label = pd.read_csv('/Users/jason/Projects/MachineLearningProjects/klanguage/train.csv')
        _1 = []
        _2 = []
        train_idx = []
        val_idx = []
        test_idx = []
        train_size = len(train_label)
        self.val_ratio = 0.2
        split_idx = int(np.floor(self.val_ratio * train_size))
        for i in train_label.values:
            _1.append(i[0])
            train_idx.append(i[1])

        train_idx, val_idx = train_idx[split_idx:], train_idx[:split_idx]

        return train_idx, val_idx

    def getLen(self, data_path, train):
        n = 0
        if train:
            for i in glob.glob(data_path + '/*'):
                n += 1
            return int(np.floor(n * (1 - self.val_ratio)))
        else:
            for i in glob.glob(data_path + '/*'):
                n += 1
            return int(np.floor(n * self.val_ratio))

    def __init__(self, root_directory, train, transform=None):
        super(CustomImageDataset, self).__init__()
        self.root_directory = root_directory
        self.transform = transform
        self.train = train
        self.train_label, self.val_label = self.read_data()
        self.val_ratio = 0.2

    def __len__(self):
        if self.train:
            return self.getLen(self.root_directory, train=True)
        else:
            return self.getLen(self.root_directory, train=False)

    def __getitem__(self, idx):
        if self.train:
            desired_width = 3
            number_str = str(idx).zfill(desired_width)
            img = Image.open(train_data_path + f'/{number_str}.jpg')
            if transforms is not None:
                img = self.transform(img)
            return img, self.train_label[idx]
        else:
            desired_width = 3
            number_str = str(idx).zfill(desired_width)
            img = Image.open(train_data_path + f'/{number_str}.jpg')
            if transforms is not None:
                img = self.transform(img)
            return img, self.val_label[idx]

In [3]:
class testCustomImage(Dataset):
    def getidx(self):
        test_idx = []
        # Needs to insert Test csv directory
        test_dir = pd.read_csv('/Users/jason/Projects/MachineLearningProjects/klanguage/test.csv')
        for i in test_dir.values:
            test_idx.append(i[0])
        return test_idx
    def getLen(self, data_path):
        n = 0
        for i in glob.glob(data_path + '/*'):
            n += 1
        return n
    def __init__(self, dir,  transform=None):
        self.dir = dir
        self.transform = transform
        self.label = self.getidx()
    def __getitem__(self, idx):
        desired_width = 3
        number_str = str(idx).zfill(desired_width)
        img = Image.open(train_data_path + f'/{number_str}.jpg')
        if transforms is not None:
            img = self.transform(img)
        return img, self.label[idx]
    def __len__(self):
        return self.getLen(self.dir)

In [4]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(32),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer5 = torch.nn.Sequential(
            torch.nn.Linear(14 * 14 * 256, 625, bias=True),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.2))
        self.layer6 = torch.nn.Linear(625, 11, bias=True)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1) 
        out = self.layer5(out)
        out = self.layer6(out)
        return out

In [5]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224), antialias=None),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224), antialias=None),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [6]:
learning_rate = 0.0005
training_epochs = 20
batch_size = 64

In [7]:
#change to appropriate directory when checking 
train_data_path = '/Users/jason/Projects/MachineLearningProjects/klanguage/train'
test_data_path = '/Users/jason/Projects/MachineLearningProjects/klanguage/test'
train_data = CustomImageDataset(train_data_path, train=True, transform=train_transform)
val_data = CustomImageDataset(train_data_path, train=False, transform=val_transform)
test_data = testCustomImage(test_data_path, transform=val_transform)
train_loader = DataLoader(train_data, batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size, shuffle=False)
test_loader = DataLoader(test_data, shuffle=False)

In [20]:
def train_loop(modelc, loss_fun, optimizerc):
    size = len(train_data)
    # modelc.to(device)
    modelc.train()
    n = 1
    start_time = time.time()
    train_loss, train_acc = 0.0, 0.0
    for batch, (X, y) in enumerate(train_loader):
        # Compute prediction and loss
        # X.to(device)
        # y.to(device)
        pred = modelc(X)
        loss = loss_fun(pred, y)
        
        optimizerc.zero_grad()
        loss.backward()
        optimizerc.step()

        # based on lab session
        train_loss += loss.item()
        train_acc += (pred.argmax(1) == y).type(torch.float).sum().item()

    train_loss /= batch_size
    train_acc /= size
    print(f"Train Error: \n Accuracy: {(100*train_acc):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    return train_loss, train_acc

In [32]:
def val_loop(modelc, loss_fun):
    print("Validation Start")
    print("---------------------")
    modelc.eval()
    size = len(val_data)
    num_batches = len(val_loader)
    validity_loss, correct = 0, 0
    validation_loss, validation_acc = 0.0, 0.0
    with torch.no_grad():
        for X, y in val_loader:

            pred = model(X)
            validation_loss += loss_fn(pred, y).item()
            validation_acc += (pred.argmax(1) == y).type(torch.float).sum().item()

    validation_loss /= batch_size
    validation_acc /= size
    print(f"Validation Error: \n Accuracy: {(100*validation_acc):>0.1f}%, Avg loss: {validation_loss:>8f} \n")

    return validation_loss, validation_acc

In [10]:
def testw(modelc):
    modelc.eval()
    list_file = list(range(len(test_data)))
    address = '/Users/jason/Projects/MachineLearningProjects/klanguage/'
    list_a=[]
    with torch.no_grad():
        for X, y in test_loader:
            pred = modelc(X)
            _, predicted = torch.max(pred, 1)
            list_a.append(predicted.tolist())
            np_list = np.array(list_a)
        submission = pd.DataFrame({'file_name': list_file , 'label': np_list.ravel()})
        submission.to_csv(path_or_buf=address + 'test2.csv')

In [11]:
model = CNN()
# model.to(device)
# print(f'device: {device}')
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.005, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()

In [76]:
for t in range(training_epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
Training loss: 2.387368  [   64/  621] at step1
Time took for step [1]: 0.29 mins
Time took for step [2]: 0.65 mins
Time took for step [3]: 1.00 mins
Time took for step [4]: 1.32 mins
Time took for step [5]: 1.66 mins
Time took for step [6]: 2.00 mins
Time took for step [7]: 2.36 mins
Time took for step [8]: 2.71 mins
Time took for step [9]: 3.05 mins
Time took for step [10]: 3.28 mins
Epoch 2
-------------------------------
Training loss: 2.109366  [   64/  621] at step1
Time took for step [1]: 0.33 mins
Time took for step [2]: 0.65 mins
Time took for step [3]: 0.98 mins
Time took for step [4]: 1.32 mins
Time took for step [5]: 1.65 mins
Time took for step [6]: 1.96 mins
Time took for step [7]: 2.29 mins
Time took for step [8]: 2.62 mins
Time took for step [9]: 2.97 mins
Time took for step [10]: 3.20 mins
Epoch 3
-------------------------------
Training loss: 2.039149  [   64/  621] at step1
Time took for step [1]: 0.32 mins
Time took for step [

In [68]:
PATH = './klanguage/'

#save model
torch.save(model, PATH + 'model.pt')  


In [ ]:
# model = torch.load(PATH + 'model.pt')
val_loop(model, loss_fn)

In [77]:
testw(model)

In [36]:
def run():
    wandb.init(
    # Set the project where this run will be logged
    project="MachineLearning2 A3",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name="experiment",
    # Track hyperparameters and run metadata
    config={
    "learning_rate": learning_rate,
    "batch_size": batch_size,
    "epochs": training_epochs
    }
    
    )
    global train_loss, train_acc, validation_loss, validation_acc
    train_loss = 0.0

    # train_acc, validation_loss, validation_acc, sample_batch, sample_img, images = 0
    model = CNN()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.005, momentum=0.9)
    loss_fn = torch.nn.CrossEntropyLoss()
    
    best_validation_acc = 0.0
    for t in range(training_epochs):
        
        print(f"Epoch {t+1}\n-------------------------------")
        train_loss, train_acc = train_loop(model, loss_fn, optimizer)
        validation_loss, validation_acc = val_loop(model, loss_fn)
        
        sample_batch, _ = next(iter(train_loader))
        sample_img = to_pil_image(sample_batch[0])
        sample_img = wandb.Image( # single image
        sample_img,
        caption="sample image"
        )
        
        images = [to_pil_image(image) for image in sample_batch[:5]] # multiple images
        
        wandb.log({
          "train_acc": train_acc,
          "train_loss": train_loss,
          "validation_acc": validation_acc,
          "validation_loss": validation_loss,
          "example": sample_img,
          "examples": [wandb.Image(image) for image in images],
          })

        if validation_acc > best_validation_acc:
            torch.save(model.state_dict(), './best.pth')

    print("Done!")
    wandb.finish()

In [37]:
run()

Epoch 1
-------------------------------
Train Error: 
 Accuracy: 10.3%, Avg loss: 0.370562 

Validation Start
---------------------
Validation Error: 
 Accuracy: 13.5%, Avg loss: 0.108317 

Epoch 2
-------------------------------
Train Error: 
 Accuracy: 19.5%, Avg loss: 0.356614 

Validation Start
---------------------
Validation Error: 
 Accuracy: 7.7%, Avg loss: 0.109169 

Epoch 3
-------------------------------
Train Error: 
 Accuracy: 26.2%, Avg loss: 0.347882 

Validation Start
---------------------
Validation Error: 
 Accuracy: 16.8%, Avg loss: 0.108808 

Epoch 4
-------------------------------
Train Error: 
 Accuracy: 27.1%, Avg loss: 0.342239 

Validation Start
---------------------
Validation Error: 
 Accuracy: 15.5%, Avg loss: 0.108474 

Epoch 5
-------------------------------
Train Error: 
 Accuracy: 30.4%, Avg loss: 0.336298 

Validation Start
---------------------
Validation Error: 
 Accuracy: 15.5%, Avg loss: 0.108591 

Epoch 6
-------------------------------
Train Error

wandb: Network error (ConnectionError), entering retry loop.


Train Error: 
 Accuracy: 45.2%, Avg loss: 0.301005 

Validation Start
---------------------
Validation Error: 
 Accuracy: 7.1%, Avg loss: 0.109765 

Epoch 12
-------------------------------
Train Error: 
 Accuracy: 48.1%, Avg loss: 0.296528 

Validation Start
---------------------
Validation Error: 
 Accuracy: 14.2%, Avg loss: 0.108060 

Epoch 13
-------------------------------
Train Error: 
 Accuracy: 47.5%, Avg loss: 0.292446 

Validation Start
---------------------
Validation Error: 
 Accuracy: 14.2%, Avg loss: 0.109244 

Epoch 14
-------------------------------


wandb: Network error (ConnectionError), entering retry loop.


Train Error: 
 Accuracy: 48.0%, Avg loss: 0.289134 

Validation Start
---------------------
Validation Error: 
 Accuracy: 10.3%, Avg loss: 0.109032 

Epoch 15
-------------------------------
Train Error: 
 Accuracy: 50.9%, Avg loss: 0.284383 

Validation Start
---------------------
Validation Error: 
 Accuracy: 11.6%, Avg loss: 0.109473 

Epoch 16
-------------------------------
Train Error: 
 Accuracy: 52.5%, Avg loss: 0.280335 

Validation Start
---------------------
Validation Error: 
 Accuracy: 14.2%, Avg loss: 0.109745 

Epoch 17
-------------------------------


wandb: Network error (ConnectionError), entering retry loop.


Train Error: 
 Accuracy: 49.3%, Avg loss: 0.276301 

Validation Start
---------------------
Validation Error: 
 Accuracy: 14.2%, Avg loss: 0.108712 

Epoch 18
-------------------------------
Train Error: 
 Accuracy: 52.3%, Avg loss: 0.271346 

Validation Start
---------------------
Validation Error: 
 Accuracy: 11.6%, Avg loss: 0.108908 

Epoch 19
-------------------------------
Train Error: 
 Accuracy: 52.2%, Avg loss: 0.268088 

Validation Start
---------------------
Validation Error: 
 Accuracy: 14.2%, Avg loss: 0.108609 

Epoch 20
-------------------------------
Train Error: 
 Accuracy: 58.5%, Avg loss: 0.263216 

Validation Start
---------------------
Validation Error: 
 Accuracy: 9.0%, Avg loss: 0.109099 

Done!


train_acc,▁▂▃▃▄▅▅▅▆▆▆▇▆▆▇▇▇▇▇█
train_loss,█▇▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁
validation_acc,▆▁█▇▇▅▆▁▆▄▁▆▆▃▄▆▆▄▆▂
validation_loss,▂▆▄▃▃▅▆▆▄██▁▆▅▇█▄▄▃▅
train_acc,0.58454
train_loss,0.26322
validation_acc,0.09032
validation_loss,0.1091


In this course, we will learn functional programming paradigm and functional programming languages (ML and C++). We will learn important concepts in programming languages, such as tail recursion, lexical scopes, currying, and lazy evaluation. We will also learn parallel programming model and the constructs for parallel programming.

NOTICE:
  * The textbooks are optional.
  * The course materials are based on those of Programming Language course in UW (by Dan Grossman).

Course Objectives:
  * Understand and internalize different programming paradigms, such as functional and object-oriented programming.
  * Develop the skills to quickly learn new programming language.
  * Learn to understand the power and elegance of programming languages and their constructs.
  * Be reasonably proficient in ML, C++, and CUDA.


Course Topics for each week:

1.	Syntax vs. semantics vs. idioms vs. libraries vs. tools 
2.	ML basics (bindings, conditionals, records, functions) 
3.	Recursive functions and recursive types 
4.	Benefits of no mutation 
5.	Algebraic datatypes, pattern matching 
6.	Tail recursion 
7.	Higher-order functions; closures 
8.	Lexical scope 
9.	Currying 
10.	Syntactic sugar 
11.	Equivalence and effects 
12.	Parametric polymorphism and container types 
13.	Type inference 
14.	Abstract types and modules 
15.	C++ some fundamentals
16.	Dynamic vs. static typing 
17.	Laziness, streams, and memoization 
18.	Implementing languages, especially higher-order functions 
19.	Macros 
20.	Eval 
21.	Abstract types via dynamic type-creation and simple contracts 
22.	Programming model for parallelism
23.	Cuda Basics
24.	Cuda Programming and applications

Learn functional programming model

Learn ML and C++ (functional paradigm)

Learn parallel programming (CUDA)
Functional Programming
Parallel Programming